# Black-Scholes-Merton Option Replication

The Black-Scholes-Merton framework is one of the foundation of quantitative finance.  Not only does it provide a theoretical pricing framework, but it also gives a practical engineering result: the manufacturing process a derivatives desk should employ to contruct options for it's customers.  This gave birth to the entire derivatives industry.

The purpose of this notebook is to explore the option manufacturing process by performing data analysis on simulated data.

## Loading Packages

Let's begin by loading the standard packages we will need.

In [1]:
##> import numpy as np
##> import pandas as pd
##> pd.options.display.max_rows = 10




Additionally, we will also require several functions from the `py_vollib` package for calculating option greeks.

In [2]:
##> from py_vollib.black_scholes_merton import black_scholes_merton
##> from py_vollib.black_scholes_merton.greeks.analytical import delta
##> from py_vollib.black_scholes_merton.greeks.analytical import vega
##> from py_vollib.black_scholes_merton.implied_volatility import implied_volatility




## Converting `py_vollib` Functions

In this section, we convert several of the functions in `py_vollib` so that they can accept the row of a `DataFrame` as their argument.  This will allow us to use these function with `DataFrame.apply()`, which will use to write compact vectorized code.  Notice that we assume a zero risk-free rate, and zero dividend yield throughout this tutorial, and thus those values are hardcoded into these functions..

In [3]:
##> def bsm_px(row):
##>     cp = row['cp']
##>     upx = row['upx']
##>     strike = row['strike']
##>     t2x = row['t2x']
##>     rf = 0
##>     volatility = row['volatility']
##>     q = 0
##>     px = black_scholes_merton(cp, upx, strike, t2x, rf, volatility, q)
##>     px = np.round(px, 2)
##>     return(px)




In [4]:
##> def bsm_delta(row):
##>     cp = row['cp']
##>     upx = row['upx']
##>     strike = row['strike']
##>     t2x = row['t2x']
##>     rf = 0
##>     volatility = row['volatility']
##>     q = 0
##>     diff = delta(cp, upx, strike, t2x, rf, volatility, q)
##>     diff = np.round(diff, 3)
##>     return(diff)




In [5]:
##> def bsm_vega(row):
##>     cp = row['cp']
##>     upx = row['upx']
##>     strike = row['strike']
##>     t2x = row['t2x']
##>     rf = 0
##>     volatility = row['volatility']
##>     q = 0
##>     vga = vega(cp, upx, strike, t2x, rf, volatility, q)
##>     vga = np.round(vga, 3)
##>     return(vga)




## Geometric Brownian Motion

The series of trade prices for a stock is often modeled as a series of random variables, which is also referred to as a *stochastic process*. There many types of stochastic processes, and some of them resemble actual stock price movements better than others.

The Black-Scholes-Merton option pricing framework assumes that the price process of the underlying asset follows a *geometric brownian motion* (GBM). This means that:

1. The price process is continuous.

1. The log return over any period of time is normally distributed.

2. The returns during any two disjoint periods are independent.

GBMs are one of the simplest types of processes that reasonably model asset price dynamics, so it's often a good place to start when learning about simulating stock price data.

The price process of a geometric brownian motion is determined by the current risk-free rate $r$ and the annualized volatility of the underlying $\sigma$.  Prices that are separated by $\Delta t$ units of time are related by following equation:

$$S_{t} =  S_{t - \Delta t} \cdot \exp\bigg(\bigg(r - \frac{1}{2}\sigma^2\bigg)\Delta t + \sigma \sqrt{\Delta t} z_{t}\bigg)$$

where $z_{t}$ is a standard normal random variable.

This is called the Euler discretization of a GBM. It will serve as the  recipe for our price-path simulation algorithm.  Note that the expression in the parentheses is log-return of the stock between time $t - \Delta t$ and $t$.

Although the  GBM assumptions are often violated in actual prices, there is still enough truth in them that the Black-Schole-Merton manufacturing process is practically useful.  It prescribes a process which tells derivative dealers how to construct the contracts that their customers are interested in.  This manufacturing process is referred to as *constructing a replicating portfolio*, which in the case of vanilla option is accomplished via dynamic trading strategy of the underlying asset.  This dynamic strategy is called *delta*-hedging. 

**Discussion Question:**  Put-call parity is a pricing identity that eminates from a static replication strategy, describe the strategy and the pricing identity.

## The Option We Will Analyze

We want to analyze what it means for a derivatives dealer to trade an and then delta-hedge the position.  Let's consider an option that actually traded in the market place:

- underlying: QQQ
- current date: 11/16/2018
- expiration: 12/21/2018
- type: put
- strike: 160
- upx: 168
- days-to-expiration (d2x): 24
- price: 2.25

From this trade price, we can calculate an implied volatility, which we will also refer to as the *pricing volatility*.  (Note that this is the typical flow of events, the price of an option is observed, and from that observed price, an implied volatility is calculated.)

In [6]:
##> pricing_vol = implied_volatility(price = 2.25, S = 168, K = 160, t = 24/252, r = 0, q = 0, flag = 'p')
##> pricing_vol = np.round(pricing_vol, 4)
##> pricing_vol




## Delta-Hedging: Single Simulated Underlying Price Path

It is typical that a derivatives dealer will trade the above option with a customer, and then hold that option option until expiration and delta hedge it on a daily basis.  

The BSM manufacturing framework states that **the dealer will *break even* if**:

1. the underlying price follows a geometric brownian motion
2. the realized volatility during the life of the option is equal to the implied volatility used to price the option
3. the dealer delta-hedges with frequent rebalancing (continuously, in the limit)

In this section we are going to explore what exactly this manufacturing process looks like for a particular price path of the underlying.  In order to do this, let's simulate a single geometric brownian motion path who's realized volatility is equal to the pricing volatility of our QQQ option.  This price path will consist of a series of daily prices that starts with 168, the spot price at the time of the trade.  We will rebalance the delta-hedge daily.

The following code generates this path:

In [7]:
##> # setting the random seed
##> np.random.seed(1)
##> 
##> # parameters of simulation
##> r = 0
##> path_vol = pricing_vol
##> dt = 1./252
##> 
##> # initializing paths
##> single_path = np.zeros(25)
##> single_path[0] = 168
##> 
##> # looping through days and generating steps in the paths
##> for t in range(1, 25):
##>     z = np.random.standard_normal(1)
##>     single_path[t] = single_path[t - 1] * np.exp((r - 0.5 * path_vol ** 2) * dt + path_vol * np.sqrt(dt) * z) # memorize this line
##>     single_path[t] = np.round(single_path[t], 2)




Let's take a look at the path we generated.  (Obviously, in a real-world situation this price path would be realized over the course of the life of the option.)

In [8]:
##> single_path



Next, let's create a `DataFrame` that will track the PNL from delta-hedging the option; this `DataFrame` will also contain all the information needed to calculate the price and greeks of the option on a daily basis.

In [9]:
##> df_path = \
##>     (
##>     pd.DataFrame(
##>         {'underlying':'QQQ',
##>          'cp':'p',
##>          'strike':160,
##>          'volatility':0.2636,
##>          'upx':single_path, 
##>          'd2x':list(range(24, -1, -1)),
##>          'buy_sell':1,
##>         }       
##>     )
##>     .assign(t2x = lambda df: df.d2x / 252)
##>     )
##> df_path




We can now use the `bsm_px()` function to calculate the prices of the option through for each day in the simulation.  At a derivative dealer, these values would be generated in real-time by a position management system.

In [10]:
##> df_path['option_price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
##> df_path




Let's calculate the deltas through time.  Notice that as the price of the underlying goes down the (absoulte) delta of the put increases, and as the price of the underlying goes up the delta decreases.

In [11]:
##> df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
##> df_path




Next, we calculate the option PNL. 

In [12]:
##> df_path['option_pnl'] = df_path['buy_sell'] * df_path['option_price'].diff()
##> df_path




Delta-hedging with daily rebalancing means at the end of each day we hold a position in the underlying who's size is equal to the negative of the delta of the option position.  Thus, the daily delta-hedging PNL is calculated as follows:

In [13]:
##> df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff() 
##> df_path




**Discussion Question:**  What is the delta-hedging position held at the end of d2x = 21.  What is the trade executed at that time?

The `total_pnl` of the delta-hedged option position is the combination of the `option_pnl` and the `delta_hedge_pnl`.

In [14]:
##> df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
##> df_path




As we can see, the total PNL of the delta-hedged option position is close to zero, but not exactly zero.

In [15]:
##> df_path['total_pnl'].sum()



**Code Challenge:**  Copy and past the above code into the space below, and then modify it to calculate the PNL for selling this option for 2.25 and then delta-hedging it over this same scenario.

## Delta-Hedging: Multiple Simulated Underlying Price Paths

As we saw in the above example, we came fairly close to a zero PNL from delta hedging on a daily basis, which is what the BSM framework suggests.  However, this was just for a single hypothetical path, so we may have just gotten lucky.  In this section, we will generate PNL data for daily delta-hedging over a variety of paths, and analyze the resulting distribution.

Let's begin by initializing our option position and scenario generation parameters.

In [16]:
##> buy_sell = -1
##> cp = 'p'
##> spot = 168.
##> strike = 160.
##> tenor = 24./252. # I want to generalize this
##> option_price = 2.25
##> pricing_vol = implied_volatility(price = option_price, S = spot, K = strike, t = tenor, r = 0, q = 0, flag = cp)
##> path_vol = pricing_vol
##> hedge_frequency = 24
##> dt = tenor / hedge_frequency  # I want to generalize this
##> r = 0
##> num_paths = 1000




Next, we initialize an array that will hold all of our paths.

In [17]:
##> multiple_paths = np.zeros((hedge_frequency + 1, num_paths))
##> multiple_paths




The first price in every path is the current spot price of the underlying.

In [18]:
##> multiple_paths[0] = spot
##> multiple_paths




Using the convenience of broadcasting in `numpy.arrays`, we can easily calculate all of the required scenarios in a few lines of code.

In [19]:
##> # setting the random seed
##> np.random.seed(1)
##> 
##> for t in range(1, hedge_frequency + 1):
##>     z = np.random.standard_normal(num_paths) 
##>     multiple_paths[t] = multiple_paths[t - 1] * np.exp((r - 0.5 * path_vol ** 2) * dt + path_vol * np.sqrt(dt) * z)
##>     multiple_paths[t] = np.round(multiple_paths[t], 2)
##>     
##> multiple_paths




Just to make sure we understand our data structures, let's pull out the first scenario and perform our delta-hedge calculations with it.  After we do that, we will wrap this code in a `for`-loop in order to perform the delta-hedge calculations for all the scenarios.  

In [20]:
##> # creating the DataFrame
##> df_path = \
##>     pd.DataFrame(
##>         {'cp':cp,
##>          'strike':strike,
##>          'volatility':path_vol,
##>          'upx':multiple_paths[:, 0], 
##>          't2x':np.linspace(tenor, 0, hedge_frequency + 1),
##>          'buy_sell':1,
##>         }       
##>     )
##> 
##> # calculating prices, greeks, and PNLs
##> df_path['option_price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
##> df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
##> df_path['option_pnl'] = df_path['buy_sell'] * df_path['option_price'].diff()
##> df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff()
##> df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
##> 
##> # viewing the path
##> df_path




Let's check the cumulative PNL for this scenario.

In [21]:
##> df_path['total_pnl'].sum()



We can now generalize the above code and perform the delta-hedging calculations on each scenario.  We will save the calculations for each scenario to analyze.

In [22]:
##> lst_scenarios = []
##> for ix_path in range(0, num_paths):
##>     
##>     # creating dataframe
##>     df_path = \
##>         pd.DataFrame(
##>             {'cp':cp,
##>              'strike':strike,
##>              'volatility':pricing_vol,
##>              'upx':multiple_paths[:, ix_path], 
##>              't2x':np.linspace(tenor, 0, hedge_frequency + 1),
##>              'buy_sell':buy_sell
##>             }
##>         )
##>     
##>     # calculating prices, greeks, and PNLs
##>     df_path['option_price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
##>     df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
##>     df_path['option_pnl'] = df_path['buy_sell'] * df_path['option_price'].diff()
##>     df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff()
##>     df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
##>     df_path['scenario'] = ix_path
##>     
##>     # storing df_path into a list
##>     lst_scenarios.append(df_path)
##> 
##> # creating a single DataFrame that contains all scenarios
##> df_all_paths = pd.concat(lst_scenarios)
##> 
##> # viewing the DataFrame
##> df_all_paths




Let's use a `.groupby()` to calculate the cummulative PNL for each scenario

In [23]:
##> df_pnl = df_all_paths.groupby(['scenario'], as_index = False)[['total_pnl']].sum()
##> df_pnl




As we can see, the average of the `total_pnls` is zero, which further demonstrates the manufacturing result of the Black-Scholes-Merton framework.

In [24]:
##> df_pnl['total_pnl'].mean()



**Discussion Queston:** If you sold this option for 0.25 more than fair-value, what would your average PNL be?

**Code Challenge:** Is the delta-hedging reducing risk?  Try to verify this with a bit of data analysis.

**Code Challenge:** Calculate the standard deviation, minimum, and maximum of the cumulative PNLs.

## What If Realized Volatility is Different that Pricing Volatility?

As we can see above, if the realized volatility of the underlying during the life of the option is equal to the pricing volatility, then the daily delta-hedging trader will break even on average (however, outcome can vary substantially depending on the scenario).

In this section, we explore what happens when realized volatility differs from pricing volatility.  In particular, we will see what happens when a trader sells an option, delta-hedges daily, but the realized volatility is 5% higher than implied.

In [25]:
##> # setting simulation parameters
##> buy_sell = -1
##> cp = 'p'
##> spot = 168.
##> strike = 160.
##> tenor = 24./252. # I want to generalize this
##> option_price = 2.25
##> pricing_vol = implied_volatility(price = option_price, S = spot, K = strike, t = tenor, r = 0, q = 0, flag = cp)
##> path_vol = pricing_vol + 0.05
##> hedge_frequency = 24
##> dt = tenor / hedge_frequency  # I want to generalize this
##> r = 0
##> num_paths = 1000
##> 
##> 
##> # initializing paths
##> multiple_paths = np.zeros((hedge_frequency + 1, num_paths))
##> multiple_paths[0] = spot
##> 
##> # setting the random seed
##> np.random.seed(1)
##> 
##> # calculating paths
##> for t in range(1, hedge_frequency + 1):
##>     z = np.random.standard_normal(num_paths) 
##>     multiple_paths[t] = multiple_paths[t - 1] * np.exp((r - 0.5 * path_vol ** 2) * dt + path_vol * np.sqrt(dt) * z)
##>     multiple_paths[t] = np.round(multiple_paths[t], 2)
##> 
##> # performing delta-hedge calculations on all the paths
##> lst_scenarios = []
##> for ix_path in range(0, num_paths):
##>     
##>     # creating the DataFrame
##>     df_path = \
##>         pd.DataFrame(
##>             {'cp':cp,
##>              'strike':strike,
##>              'volatility':pricing_vol,
##>              'upx':multiple_paths[:, ix_path], 
##>              't2x':np.linspace(tenor, 0, hedge_frequency + 1),
##>              'buy_sell':buy_sell
##>             }
##>         )
##>     
##>     # calculating prices, greeks, and PNLs
##>     df_path['option_price'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_px, axis = 1)
##>     df_path['delta'] = df_path[['cp', 'upx', 'strike', 't2x', 'volatility']].apply(bsm_delta, axis = 1)
##>     df_path['option_pnl'] = df_path['buy_sell'] * df_path['option_price'].diff()
##>     df_path['delta_hedge_pnl'] = -df_path['buy_sell'] * df_path['delta'].shift(1) * df_path['upx'].diff()
##>     df_path['total_pnl'] = df_path['option_pnl'] + df_path['delta_hedge_pnl']
##>     df_path['scenario'] = ix_path
##>     
##>     # storing df_path into a list
##>     lst_scenarios.append(df_path)
##>     
##> # creating a single DataFrame that contains all scenarios    
##> df_all_paths = pd.concat(lst_scenarios)
##> 
##> # calculating cumulative PNLs
##> df_pnl = df_all_paths.groupby(['scenario'], as_index = False)[['total_pnl']].sum()

As we can see, since realized is greater than implied, we lose money on average.

In [26]:
##> df_pnl['total_pnl'].mean()

**Code Challenge:** Use `bsm_vega` and verify that this PNL is consistent with the identity: $vega * (implied - realzed)$.

## Increasing Delta-Hedge Frequency Reduces PNL Variability

The BSM manufacturing framework states that in limit of continuous delta-hedging, that these results become deterministic.  The means that the delta-hedging outcomes are always the same for each scenario.  Your final code challenge is to explore this via data analysis.

**Code Challenge:** Copy and paste the above code, and see what happens to the dispersion of the distribution of the delta-hedge PNL outcomes when you double and quadruple the `hedge_frequency`.